In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Análisis exploratorio + Pre-procesamiento - Dataset HIBA

In [ ]:
os.makedirs('./ISIC_HIBASkinLesionsData', exist_ok=True)

In [ ]:
data = pd.read_csv('./ISIC_HIBASkinLesionsData/metadata.csv')
data['image_type'] = data['image_type'].str.replace('clinical: close-up','clinical').replace('clinical: overview','clinical')
data_dermoscopic = data[data['image_type'] == 'dermoscopic']
data_dermoscopic

In [ ]:
# Eliminamos las columnas que no son de interés para nuestro análisis
columnas_a_eliminar = ['copyright_license','concomitant_biopsy','dermoscopic_type', 'diagnosis_confirm_type', 'family_hx_mm', 'fitzpatrick_skin_type', 'image_type', 'personal_hx_mm']
data_dermoscopic = data_dermoscopic.drop(columnas_a_eliminar, axis=1)
data_dermoscopic

In [ ]:
# Comprobación de valores null por columna
valores_nulos_por_columna = data_dermoscopic.isnull().sum()
print(valores_nulos_por_columna)

In [ ]:
info_df = data_dermoscopic.info()
print(info_df)

In [ ]:
lesions_per_patient = []
images_per_patient = []
derm_per_patient = []

for pat in data_dermoscopic.patient_id.unique():
  aux = data_dermoscopic[data_dermoscopic.patient_id==pat]
  lesions_per_patient.append(len(aux.lesion_id.unique()))
  images_per_patient.append(len(aux))

lesions_per_patient = np.asarray(lesions_per_patient)
images_per_patient = np.asarray(images_per_patient)

In [ ]:
# Eliminamos duplicados
patient_data = data_dermoscopic[['patient_id','age_approx', 'sex']].reset_index(drop=True) 
patient_data.drop_duplicates(['patient_id'],inplace=True)
patient_data.reset_index(inplace=True,drop=True)
patient_data

In [ ]:
# Se reemplazan valores vacíos en la columna 'age_approx' con NaN
patient_data['age_approx'] = patient_data['age_approx'].astype(float)

# Contamos los valores de 'age_approx', incluyendo NaN
pvc = patient_data['age_approx'].value_counts(dropna=False)

age_min = patient_data["age_approx"].min()
age_max = patient_data["age_approx"].max()
age_mean = patient_data["age_approx"].mean()
age_std = patient_data["age_approx"].std()

print("Estadísticas de edad de los pacientes:")
print(f"Mínimo: {age_min}")
print(f"Máximo: {age_max}")
print(f"Media: {age_mean}")
print(f"Desviación estándar: {age_std}")

sns.set(style="whitegrid", palette="pastel")
sns.histplot(data=patient_data, x='age_approx', bins=30, kde=True, color='#6AB7F9')
plt.xlabel('Edad Aproximada')
plt.ylabel('Número de Pacientes')
plt.title('Distribución de Edades de los Pacientes')
plt.show()

In [ ]:
# Filtramos datos por género
female_px = patient_data[patient_data.sex == 'female']
male_px = patient_data[patient_data.sex == 'male']

# Configuramos estilo y paleta de colores de Seaborn
sns.set(style="whitegrid", palette="pastel")

# Creamos histogramas separados por género
sns.histplot(data=female_px, x='age_approx', color='#FABCB7', label='mujer', kde=True)
sns.histplot(data=male_px, x='age_approx', color='#B7D5FA', label='hombre', kde=True)

plt.xlabel('Edad Aproximada')
plt.ylabel('Número de Pacientes')
plt.title('Distribución de Edades de los Pacientes por Género')
plt.legend()
plt.show()

In [ ]:
lesion_data = data[['patient_id','lesion_id','benign_malignant', 'diagnosis','diagnosis_confirm_type', 'anatom_site_general']]
lesion_data.head()

In [ ]:
# Eliminamos duplicados
lesion_data.drop_duplicates(['patient_id', 'lesion_id'], inplace=True)
len(lesion_data)

In [ ]:
data_dermoscopic.diagnosis.value_counts()

In [ ]:
# Eliminamos las filas donde el valor en la columna 'diagnosis' es NaN
data_dermoscopic = data_dermoscopic.dropna(subset=['diagnosis'])

In [ ]:
# Definimos un diccionario de mapeo de valores específicos de 'Diagnosis' a 'benigno' o 'maligno'
mapeo_diagnosis = {
    'basal cell carcinoma': 'carcinoma',
    'squamous cell carcinoma': 'carcinoma',
    'seborrheic keratosis': 'keratosis',
    'actinic keratosis': 'keratosis',
    'nevus': 'nevus',
    'melanoma':'melanoma',
    'dermatofibroma':'dermatofibroma',
    'solar lentigo':'solar lentigo',
    'vascular lesion': 'vascular lesion'

}

# Aplicamos el mapeo a la columna 'Diagnosis' y asigna los resultados a una nueva columna llamada 'Tipo'
data_dermoscopic['diagnosis'] = data_dermoscopic['diagnosis'].map(mapeo_diagnosis)

In [ ]:
count_data = data_dermoscopic['diagnosis'].value_counts().reset_index()
count_data.columns = ['Diagnosis', 'Count']
ax = sns.barplot(data=count_data, x='Count', y='Diagnosis', hue='Diagnosis', dodge=False, legend=False)
plt.xlabel('Número de pacientes')
plt.ylabel('Diagnóstico')
plt.title('Pacientes según el tipo de diagnóstico')
plt.show()

In [ ]:
count_data = data_dermoscopic['benign_malignant'].value_counts().reset_index()
count_data.columns = ['Diagnosis', 'Count']

# Count de casos benignos y malignos
num_benign = count_data[count_data['Diagnosis'] == 'benign']['Count'].values[0]
num_malignant = count_data[count_data['Diagnosis'] == 'malignant']['Count'].values[0]

ax = sns.barplot(data=count_data, x='Diagnosis', y='Count', hue='Diagnosis', dodge=False, legend=False)
plt.xlabel('Tipo')
plt.ylabel('Count')
plt.title('Ratio de benigno versus maligno\nBenignos: {} | Malignos: {}'.format(num_benign, num_malignant))
plt.show()

In [ ]:
lesion_data.anatom_site_general = lesion_data.anatom_site_general.replace('', np.nan)
lesion_data.anatom_site_general.value_counts(dropna=False)

In [ ]:
#Eliminamos las muestras de tipo Nan para anatom_site_general y age
data_dermoscopic = data_dermoscopic.dropna(subset=['anatom_site_general'])
data_dermoscopic = data_dermoscopic.dropna(subset=['age_approx'])
data_dermoscopic = data_dermoscopic.dropna(subset=['sex'])

data_dermoscopic

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x='diagnosis', hue='anatom_site_general', data=data_dermoscopic, palette='pastel')
for i,v in enumerate(lesion_data.anatom_site_general.unique()):
  try:
    ax.bar_label(ax.containers[i])
  except:
    continue

In [ ]:
data_dermoscopic = data_dermoscopic[data_dermoscopic['diagnosis'] != 'lichenoid keratosis']

In [ ]:
data_dermoscopic

In [ ]:
# Eliminamos columnas no necesarias
columns_to_drop = ['lesion_id', 'patient_id', 'attribution']
data_dermoscopic.drop(columns=columns_to_drop, inplace=True)  

In [ ]:
# Categorizamos las variables para poder utilizarlas en el modelo 
sex_mapping = {'male': 0, 'female': 1}

diagnosis_mapping = {'nevus': 0,                      
'basal cell carcinoma': 1,        
'melanoma': 2,                   
'squamous cell carcinoma': 3,    
'seborrheic keratosis': 4,        
'actinic keratosis': 5,           
'vascular lesion': 6,             
'dermatofibroma': 7,              
'solar lentigo': 8,               
'lichenoid keratosis': 9 }  

benign_malignant_mapping = {'benign': 0, 'malignant': 1}

anatom_site_general_mapping = {
'head/neck': 0,          
'posterior torso': 1,    
'lower extremity': 2,  
'anterior torso': 3,   
'upper extremity': 4,    
'palms/soles': 5,         
'lateral torso': 6,        
'oral/genital': 7 }  

data_dermoscopic['sex'] = data_dermoscopic['sex'].replace(sex_mapping)

data_dermoscopic.rename(columns={'isic_id': 'id'}, inplace=True)
data_dermoscopic.rename(columns={'age_approx': 'age'}, inplace=True)
data_dermoscopic['age'] = data_dermoscopic['age'].astype(int)
data_dermoscopic['sex'] = data_dermoscopic['sex'].astype(int) 

In [ ]:
data_dermoscopic

In [ ]:
# Ahora obtenemos una lista de los nombres de archivos correspondientes a las imágenes dermoscópicas
archivos_dermoscopic = data_dermoscopic['id'].tolist()
archivos_dermoscopic = [id + ".JPG" for id in archivos_dermoscopic]

# Ruta de la carpeta que contiene todas las imágenes
carpeta = './ISIC_HIBASkinLesionsData/Dataset/'

# Obtenemos una lista de todos los archivos en la carpeta
archivos_totales = os.listdir(carpeta)

In [ ]:
import shutil

# Si el archivo no es imagen dermatoscopica y existe lo eliminamos de la carpeta de imágenes
for archivo in archivos_totales:
    if archivo not in archivos_dermoscopic:   
        ruta_completa = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_completa):
            os.remove(ruta_completa)
    else:
        # Obtenemos la fila específica del dataset para el archivo actual
        nombre_sin_extension, extension = os.path.splitext(archivo)
        fila_especifica = data_dermoscopic.loc[data_dermoscopic['id'] == nombre_sin_extension]

        # Obtenemos la etiqueta de la fila
        etiqueta = fila_especifica['benign_malignant'].values[0]  
        
        # Mover el archivo según la etiqueta
        if etiqueta == 'benign':
            shutil.copy(os.path.join(carpeta, archivo), "./ISIC_HIBASkinLesionsData/Dataset/Benigno/")
        elif etiqueta == 'malignant':
            shutil.copy(os.path.join(carpeta, archivo), "./ISIC_HIBASkinLesionsData/Dataset/Maligno/")

In [ ]:
data_dermoscopic['benign_malignant'] = data_dermoscopic['benign_malignant'].replace(benign_malignant_mapping)

In [ ]:
# Añadimos los factores climatoloógicos obtenidos anteriormente al conjunto

#Temperatura
data_dermoscopic['max_temp'] = 36.0
data_dermoscopic['min_tmp'] = -2.2
data_dermoscopic['mean_temp'] = 17.16158674803836
data_dermoscopic['median_temp'] = 17.0
data_dermoscopic['std_temp'] = 7.031212802223093

#Dew Point
data_dermoscopic['max_dew_point'] = 25.1
data_dermoscopic['min_dew_point'] = -10.8
data_dermoscopic['mean_dew_point'] = 11.131390885616582
data_dermoscopic['median_dew_point'] = 12.0
data_dermoscopic['std_dew_point'] = 6.1068307450132835

#Wind Speed
data_dermoscopic['max_wind_speed'] = 250.02
data_dermoscopic['min_wind_speed'] = 0.0
data_dermoscopic['mean_wind_speed'] = 12.097432691302986
data_dermoscopic['median_wind_speed'] = 11.16
data_dermoscopic['std_wind_speed'] = 7.027559785092077

#Wind Direction
data_dermoscopic['max_wind_direction'] = 360.0
data_dermoscopic['min_wind_direction'] = 7.0
data_dermoscopic['mean_wind_direction'] = 166.10650887573965
data_dermoscopic['median_wind_direction'] = 140.0
data_dermoscopic['std_wind_direction'] = 101.95115122631336

#Visibility
data_dermoscopic['max_visibility'] = 21.02
data_dermoscopic['min_visibility'] = 0.0
data_dermoscopic['mean_visibility'] = 8.779571632390299
data_dermoscopic['median_visibility'] = 10.0
data_dermoscopic['std_visibility'] = 2.6558069047881507

In [ ]:
data_dermoscopic = data_dermoscopic.dropna(subset=['diagnosis'])

In [ ]:
# Leemos el nuevo dataset de calculos solares
data_calcs = pd.read_csv('./ISIC_HIBASkinLesionsData/HIBA_solar_calcs.csv')
data_calcs

In [ ]:
# Transformamos el nuevo dataset para obtener una estructura entendible y facilmente iterable 
variables = [
    "ALLSKY_KT",
    "CLOUD_AMT",
    "TOA_SW_DWN",
    "ALLSKY_SFC_UVA",
    "ALLSKY_SFC_UVB",
    "ALLSKY_SRF_ALB",
    "ALLSKY_SFC_SW_DNI",
    "ALLSKY_SFC_SW_DWN",
    "ALLSKY_SFC_PAR_TOT",
    "ALLSKY_SFC_SW_DIFF"
]

columns = [
    "max",
    "min",
    "mean",
    "median",
    "std",
]

data_calcs.index = variables
data_calcs.columns = columns

data_calcs

In [ ]:
# Introducimos los nuevos datos al dataset original
for index, row in data_calcs.iterrows():
    nombre = row.name  
    for columna in data_calcs.columns:
        estadistica = columna 
        nuevo_nombre_columna = f"{nombre}_{estadistica}"  
        data_dermoscopic[nuevo_nombre_columna] = row[columna]

In [ ]:
data_dermoscopic

In [ ]:
# Guardamos el dataset de factores externos
data_dermoscopic.to_csv('./ISIC_HIBASkinLesionsData/final_metadata_HIBA.csv', index=False)